In [1]:
import os
from pathlib import Path

while Path.cwd().name.lower() != "aiml25-exam" and "aiml25-exam" in str(Path.cwd()).lower():
    os.chdir("..")  # Move up one directory
print(f"Working directory set to: {Path.cwd()}")

Working directory set to: /Users/isimisi/Documents/GitHub/isimisi/AIML25-Exam


In [2]:
from src.yolo.yolo import Yolo
from ultralytics import YOLO
from src.utils.path import from_root

In [3]:
model = YOLO(from_root("yolov8m-seg-best.pt"))

yolo = Yolo(model)
yolo.predict(from_root("datasets/test/images/1.png"))


image 1/1 /Users/isimisi/Documents/GitHub/isimisi/AIML25-Exam/datasets/test/images/1.png: 448x640 24 sub-flows, 768.1ms
Speed: 1.5ms preprocess, 768.1ms inference, 8.5ms postprocess per image at shape (1, 3, 448, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'sub-flow'}
 obb: None
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 2

In [4]:
bboxes = yolo.getBBoxes()
bboxes

[{'xywh': [637.135498046875,
   839.093994140625,
   670.6090087890625,
   769.81201171875],
  'confidence': 0.9916790127754211,
  'class_id': 0,
  'xyxy': [301.83099365234375, 454.1880187988281, 972.4400024414062, 1224.0]},
 {'xywh': [1155.082763671875,
   307.9461975097656,
   1162.8863525390625,
   572.1382446289062],
  'confidence': 0.9562817811965942,
  'class_id': 0,
  'xyxy': [573.6396484375,
   21.877056121826172,
   1736.5260009765625,
   594.0153198242188]},
 {'xywh': [1512.298095703125,
   291.6260986328125,
   495.0123291015625,
   361.64898681640625],
  'confidence': 0.9480721950531006,
  'class_id': 0,
  'xyxy': [1264.7918701171875,
   110.80159759521484,
   1759.80419921875,
   472.4505920410156]},
 {'xywh': [1498.63232421875,
   383.5832824707031,
   518.131103515625,
   586.2085571289062],
  'confidence': 0.9210164546966553,
  'class_id': 0,
  'xyxy': [1239.5667724609375,
   90.47899627685547,
   1757.6978759765625,
   676.6875610351562]},
 {'xywh': [1143.7879638671875

In [5]:
images = yolo.cropImages(bboxes)

In [6]:
os.makedirs("tmp", exist_ok=True)

for i, img in enumerate(images):
    filename = f"tmp/image_{i}.png"
    img.save(from_root(filename))

In [7]:
watson_prompt = """
Objective
You are a highly intelligent AI specialized in image recognition and data transformation of legal structures. 
Your task is to analyze an input image containing flowchart elements (nodes and edges) and transform the recognized content into a structured JSON format of the edges.

Image Content:
    Valid Nodes may include: shapes, text, or small illustrations.
    Valid Edges describe relationships (e.g., ownership or connection type) and are represented as lines/arrows between nodes.

Edges must follow this JSON structure:

{
"source": string (node name of source),
"target": string (node name of target)
}

Find edges that are definitively connected to nodes. If the edge runs out of the image, do not include the connection.
"""

In [ ]:
sizes = []
for box in bboxes:
    [x1, y1, x2, y2] = box['xyxy']
    w = x2 - x1
    h = y2 - y1
    sizes.append((w * h))


print(sizes)

[173192.50946890004, 179020.71105785388, 242522.49235937372, 291046.1721273307, 303732.8905485291, 316142.5100163333, 338601.74941073265, 341776.7980471216, 417520.97063058615, 516242.8496672623, 558059.8823130354, 591116.942103755, 631043.7169083161, 631831.8853492248, 665331.7786249076, 724930.9030533456, 735119.8073117535, 741277.8038098225, 862906.4359912006, 865908.3761592708, 927056.8916977532, 1077133.2846016623, 1177228.2864189297, 1471326.113259608]
